# Erster Agent

Es ist an der Zeit einen Agenten zu erstellen. Und zwar werden wir einen Agenten mit zwei-drei Tools erstellen. 
Als erstes Tool möchten wir dem Agenten Zugriff auf Wikipedia geben, damit der Agent mithilfe der Wikipedia-Artikel fundierte Antworten geben kann.
Als zweites Tool bekommt der die DuckDuckGo-Funktion.



In [2]:
!pip install langchain langchain_openai python-dotenv duckduckgo-search wikipedia

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached soupsieve-2.5-py3-none-any.whl.metadata (4.7 kB)
Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)
Using cached soupsieve-2.5-py3-none-any.whl (36 kB)
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=7a42f602fb0e877e13af2670ecbbeaf61763490aa6ef061aad14c80961ee26a6
  Stored in directory: /Users/sgruenheit/Library/Caches/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia

[notice] A new release of pip is available: 24.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [16]:
import os
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain.prompts import SystemMessagePromptTemplate, ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate, PromptTemplate

from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_community.tools import DuckDuckGoSearchResults
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.prompts import ChatPromptTemplate

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

model = ChatOpenAI(model="gpt-4o")

In [11]:
search = DuckDuckGoSearchResults()

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

tools = [search, wikipedia]

In [17]:
system_template = "Du bist ein hilfreicher Assistent, der in der Lage ist die notwendigen Fakten im Internet zu suchen oder bei Wikipedia nachzuschlagen."

prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template=system_template)),
        MessagesPlaceholder(variable_name='chat_history', optional=True),
        HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
        MessagesPlaceholder(variable_name='agent_scratchpad')
    ]
)

from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(model, tools, prompt_template)

In [18]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [19]:
result = agent_executor.invoke({"input": "Guten Tag! Wann wurde Budeskanuzler Olfa Scholz geboren?"})

result



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Olaf Scholz'}`


Page: Olaf Scholz
Summary: Olaf Scholz (German: [ˈoːlaf ˈʃɔlts] ; born (1958-06-14)14 June 1958) is a German politician who has been the chancellor of Germany since 8 December 2021. A member of the Social Democratic Party (SPD), he previously served as Vice Chancellor in the fourth Merkel cabinet and as Federal Minister of Finance from 2018 to 2021. He was also First Mayor of Hamburg from 2011 to 2018, deputy leader of the SPD from 2009 to 2019, and Federal Minister of Labour and Social Affairs from 2007 to 2009.
Scholz began his career as a lawyer specialising in labour and employment law. He became a member of the SPD in the 1970s and was a member of the Bundestag from 1998 to 2011. Scholz served in the Hamburg Government under First Mayor Ortwin Runde in 2001 and became General Secretary of the SPD in 2002, where he served alongside SPD leader and then-Chancellor Gerhard Schröder. He bec

{'input': 'Guten Tag! Wann wurde Budeskanuzler Olfa Scholz geboren?',
 'output': 'Olaf Scholz wurde am 14. Juni 1958 geboren.'}

In [20]:
result = agent_executor.invoke({"input": "Wann findet das Münchner Oktoberfest statt?"})

result



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_results_json` with `{'query': 'Münchner Oktoberfest 2023 Termine'}`


[snippet: Wann beginnt das Oktoberfest in München? Das 188. Oktoberfest auf der Theresienwiese beginnt am Samstag, den 16.September 2023.Um Punkt 12 Uhr zapft der Münchner Oberbürgermeister Dieter Reiter ..., title: Oktoberfest 2023 in München: Öffnungszeiten und Termine auf der Wiesn, link: https://www.sueddeutsche.de/muenchen/oktoberfest-2023-muenchen-oeffnungszeiten-termine-1.5642157], [snippet: Oktoberfest 2023: Der Bierpreis von bis zu 14,90 Euro kann sich sehen lassen. Aber das hindert kaum einen daran, mehrere Liter zu verköstigen. Quelle: imago images/ingimage. Schnäppchen sucht man auf dem Oktoberfest vergebens. Aber es gibt einige Mittagsangebote, den Feiertag am Dienstag, 3., title: Oktoberfest 2023 in München: Datum, Bier, Preise, Wetter, Tipps, link: https://www.reisereporter.de/reisenews/oktoberfest-2023-in-muenchen-datum-bier-preise-wetter-

{'input': 'Wann findet das Münchner Oktoberfest statt?',
 'output': 'Das Münchner Oktoberfest 2023 findet vom 16. September bis zum 3. Oktober statt.'}

## ReAct

Ein _ReAct Agent_ ist ein KI-System, das "Reasoning and Acting" (Denken und Handeln) kombiniert.

- Abwechslung von Denken und Handeln: Der Agent wechselt zwischen Denkphasen, in denen er plant und schlussfolgert, und Handlungsphasen, in denen er Aktionen ausführt.
- Schrittweise Problemlösung: Komplexe Aufgaben werden in kleinere Schritte zerlegt und sequentiell bearbeitet.
- Anpassungsfähigkeit: Der Agent kann seinen Ansatz basierend auf Zwischenergebnissen oder neuen Informationen anpassen.
- Werkzeuge: ReAct Agenten können oft externe Tools oder Datenquellen in ihren Problemlösungsprozess einbinden.
